In [1]:
using SpecialFunctions, IterTools, HardSphereDynamics, StatsBase # For Std Error.

┌ Info: Precompiling HardSphereDynamics [2cbf6e9d-eec9-4262-aa18-cb9280186e6b]
└ @ Base loading.jl:1278


Aqui abajo Rosa define todos los resultados téoricos para cualquier número de dimensiones, siempre
que no haya "sombras prohibidas" (es decir, que las bolas siempre quepan unas al lado de otras).

In [2]:

function SphereArea(n)
    return 2*(pi^(n/2))/gamma(n/2)
end

function NormalizationFactor(n, balls=2)
    d = 2*n
    return SphereArea(d)/SphereArea(d-1)*(d-1)
end

function Ak(boxdims,m)
    result = 0
    for aks in subsets(boxdims, m)
        result += prod(aks)
    end
    return result
end

function AvailableVolume(n, boxdims, r)
    vol = prod((2*boxdims).^2)
    for k in 0:n
        vol -= (4^n)*(pi)^(n/2-k/2)*r^(k+n)/gamma(k/2+n/2+1)*Ak(boxdims, n-k)*(-1)^k
    end
    return vol
end

function SphereCollisionArea(n, boxdims, r)
    area = 0 
    for k in 0:n
        area += 2^(2*n+1/2)*(pi^(n/2-k/2))*(r^(k+n-1))/gamma(k/2+n/2)*Ak(boxdims, n-k)*(-1)^k
    end
    return area
end

function WallCollisionArea(n, boxdims, r, dim)
    otherboxdims = deleteat!(copy(boxdims),dim)
    area = prod((2*otherboxdims).^2)*(2*boxdims[dim])
    for k in 0:(n-1)
        area -= 2^(2*n-2)*(pi^(n/2 - k/2))*(r^(k+n))/gamma(k/2+n/2+1)*(-1)^k*Ak(otherboxdims, n-1-k)
    end
    return area
end

function SphereCollisionTime(n, boxdimsh, r)
    boxdimsa = boxdimsh/2 .- r 
    return NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/SphereCollisionArea(n, boxdimsa, r)
end

function WallCollisionTime(n, boxdimsh, r, dim)
    boxdimsa = boxdimsh/2 .- r 
    return NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/WallCollisionArea(n, boxdimsa, r, dim)
end
    
function HoppingCollisionTime(n, boxdimsh, r, dim)
    boxdimsa = boxdimsh/2 .- r 
    otherboxdims = deleteat!(copy(boxdimsa),dim)
    #if r < minimum(boxdimsh)/4
        result= NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/
           AvailableVolume(n-1, otherboxdims, r)/2^(2.5)/boxdimsa[dim]
    #else
    #    result=1.0
    #end
    
end

function CollisionTimesTheo(r, n=3, boxdimsh = [1.0, 1.5, 1.75])
    wall_times = []
    hop_times = []
    for dim in 1:n
        w = WallCollisionTime(n, boxdimsh, r, dim)
        push!(wall_times, w, w)
        push!(hop_times, HoppingCollisionTime(n, boxdimsh, r, dim))
    end
    result = hcat(r , SphereCollisionTime(n, boxdimsh, r), wall_times..., hop_times...)
    return result
end

CollisionTimesTheo (generic function with 3 methods)

In [21]:
using Parameters, Statistics, HardSphereDynamics, StaticArrays

function BallsRelativeSign(state)
    b1, b2 = state
    return broadcast(sign, b1.x - b2.x)
end

function mean_interval(times)
    return mean(diff(times))
end

h1 = 1.0
h2 = 1.75
h3 = 1.33
r = 0.1
rmax=  (h1+h3-√(2*h1*h3))/2   #maximal possible radius hopping in h2 direction

Nmax=10^8

function CollisionTimesSim(r, h1 = h1, h2 = h2, h3 = h3 ,num_collisions = Nmax)
    table = HardSphereDynamics.RectangularBox(SA[-h1/2, -h2/2, -h3/2],
                                              SA[+h1/2, +h2/2, +h3/2])

    n = 3
    spheres = 2
    fluid = HardSphereFluid{n,Float64}(table, spheres, r)
    initial_condition!(fluid.balls, table)

    # set up simulation:
    collision_type = ElasticCollision()
    flow_type = ExternalFieldFlow(SA[0.0, 0.0, 0.0])
    event_handler = AllToAll(fluid, flow_type)

    simulation =  HardSphereSimulation(
        fluid, event_handler, flow_type, collision_type);

    @unpack fluid, flow_dynamics, collision_dynamics, event_handler = simulation
    states = [deepcopy(fluid.balls)]
    times = [0.0]
    collision_types = [:none]
    partner2 = [0] 
    partner1 = [0]
    hopping_times = []
    hopping_directions = []
    for i in 1:num_collisions
        HardSphereDynamics.flow!(fluid.balls, (event_handler.next_collision_time - simulation.current_time), 
            flow_dynamics)
        simulation.current_time = event_handler.next_collision_time
        push!(collision_types, event_handler.collision_type)
        push!(partner2, event_handler.partner2)
        push!(partner1, event_handler.partner1)
        HardSphereDynamics.collide!(fluid, event_handler, collision_dynamics)
        HardSphereDynamics.find_collision!(event_handler, fluid, flow_dynamics)
        push!(states, deepcopy(fluid.balls))
        push!(times, simulation.current_time)
        prev, current = states[i:i+1]
        signprev, signcurrent = broadcast(BallsRelativeSign,[prev, current])
        if any(signprev .!= signcurrent)
            b1, b2 = states[i]
            start_time, final_time = times[i:i+1]
            hop_times = start_time .+( b1.x - b2.x)./(b2.v - b1.v)
            for (dir, time) in enumerate(hop_times)
                if all([time>=start_time, time<=final_time])
                    push!(hopping_times, time)
                    push!(hopping_directions, dir)
                end
            end
        end
    end
    
    wall_idx = (collision_types.== :wall_collision).&(partner1.== 1)
    wall_times = times[wall_idx];
    which_wall = partner2[wall_idx];
    wall_means = []
    
    # The std errros are no longer missin!
    wall_errors = []
    hop_errors  = []
    
    for i in 1:length(fluid.box.walls)
        
        enparedcorrecta=wall_times[which_wall .== i]
        error=sem(diff(enparedcorrecta))
        nn=length(enparedcorrecta)
     #   println( " En la pared $i, chocamos $nn veces")
        
        push!(wall_means,mean_interval(enparedcorrecta))
        push!(wall_errors, error)
        
    end
    
    hop_means = []
    hop_errors= []

    for i in 1:n
        
        cual_hop=hopping_times[hopping_directions .== i]
        ll=length(cual_hop)
        if ll > 2
            error=sem(diff(cual_hop))
        println("en el hop direccion $i, hopamos $ll veces con radio $r")
            push!(hop_means, mean_interval(cual_hop))
            push!(hop_errors, error)
        else
            push!(hop_means, NaN)
            push!(hop_errors, NaN)
        end
        
    end
    
    colision=times[collision_types.== :disc_collision]
    meandisc=mean_interval(colision)
    errorcol=sem(diff(colision))
    
    
    result=hcat(r, meandisc, errorcol,wall_means..., wall_errors..., hop_means..., hop_errors...)
    
    return result
        
end

CollisionTimesSim (generic function with 5 methods)

In [22]:
rmax

0.34952467848499547

In [23]:
@time chin, pun, cuaz, el, eslam = CollisionTimesSim(0.21)

en el hop direccion 1, hopamos 23901568 veces con radio 0.21
en el hop direccion 2, hopamos 6462456 veces con radio 0.21
en el hop direccion 3, hopamos 12782843 veces con radio 0.21
1664.642725 seconds (2.10 G allocations: 132.945 GiB, 46.96% gc time)


1×21 Array{Float64,2}:
 0.21  2.59037  0.000698368  3.13187  …  0.000249759  0.0022214  0.000714641

In [24]:
rmax

0.34952467848499547

In [26]:
deltaslog=-10:0.25:-2
rs=rmax.-exp.(deltaslog)

33-element Array{Float64,1}:
 0.349479278555233
 0.3494663838212646
 0.34944982665510776
 0.34942856683293405
 0.34940126868090876
 0.3493662171598797
 0.3493212101159848
 0.3492634199276938
 0.34918921585709295
 0.3490939359444198
 0.3489715941148476
 0.34881450409615294
 0.34861279651944094
 ⋮
 0.3408729832818748
 0.33841568194675314
 0.3352604445759962
 0.3312090395962613
 0.32600693262898633
 0.319327295062677
 0.31075047065327344
 0.2997376101171315
 0.2855968172782879
 0.2674396798610967
 0.24412545392313112
 0.21418939524838276

In [27]:
theo = vcat(broadcast(CollisionTimesTheo, rs)...);
sim = vcat(broadcast(CollisionTimesSim, rs)...);

en el hop direccion 1, hopamos 22351751 veces con radio 0.349479278555233
en el hop direccion 3, hopamos 8352572 veces con radio 0.349479278555233
en el hop direccion 1, hopamos 22327418 veces con radio 0.3494663838212646
en el hop direccion 3, hopamos 8363451 veces con radio 0.3494663838212646
en el hop direccion 1, hopamos 22345740 veces con radio 0.34944982665510776
en el hop direccion 3, hopamos 8366437 veces con radio 0.34944982665510776
en el hop direccion 1, hopamos 22337151 veces con radio 0.34942856683293405
en el hop direccion 3, hopamos 8361491 veces con radio 0.34942856683293405
en el hop direccion 1, hopamos 22311408 veces con radio 0.34940126868090876
en el hop direccion 3, hopamos 8355779 veces con radio 0.34940126868090876
en el hop direccion 1, hopamos 22336346 veces con radio 0.3493662171598797
en el hop direccion 3, hopamos 8352526 veces con radio 0.3493662171598797
en el hop direccion 1, hopamos 22323257 veces con radio 0.3493212101159848
en el hop direccion 3, hopa

In [13]:
using JLD

In [28]:
nota="Los Resultados Teoricos y Simulados estan en dos tablas.
Prmero las bolas entre ellas.
Luego vienen en Teoricas los golpes con las paredes de un bola 
contra una pared asi: 
Columna   Pared
3         x positiva
4         x negativa
5         y positiva
6         y negativa
7         z positiva
8         z negativa
Luego vienen los hops, 
9         hop x
10         hop y
11        hop z
En las simulaciones, entre cada tipo de eventos, en el mismo orden, hay una grupo de columnas 
de errores estandar. Es decir, despues de choques entre bolas, una columna de error. Despues
de seis choques en paredes, vienen seis columnas de error. Y despues de tres columnas de hops, viene
tres columnas de error. 
"

otranota="La variable h contiene los tamaños de la caja. 
La variable Nmax el numero de collisiones total.
La variable rmax es el radio limite para cruzar en la direccion h2 (hopping in y direction)"
onde="hoppingclosetozero3D-1e8-20210218.jld"

h=(h1,h2,h3)

save(onde, "nota01", nota, "nota02", otranota, 
    "teoricas" , theo, "numericas", sim, "h", h, "Nmax", Nmax, "rmax", rmax)


In [ ]:
sim[:,17]

In [ ]:
using Plots
scatter(rs,sim[:,2])
plot!(rs,theo[:,2])
title!("Disc collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,sim[:,4])
plot!(rs,theo[:,3])
title!("X wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,sim[:,6])
plot!(rs,theo[:,5])
title!("y wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,sim[:,8])
plot!(rs,theo[:,7])
title!("z wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,sim[:,16] )
plot!(rs, theo[:, 9])
title!("Hopping x")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,sim[:,17])
plot!(rs, theo[:,10])
title!("Hopping y")
xlabel!("r")
ylabel!("mean time")

In [ ]:
savefig("hop_axis_2_3D_inset.pdf")

In [ ]:
scatter(rs,sim[:,18] )
plot!(rs,theo[:,11])
title!("Hopping z")
xlabel!("r")
ylabel!("mean time")

A box with different dimensions

In [ ]:
h1 = 1
h2 = 2
h3 = 3
theo2 = vcat(broadcast(function (r)  CollisionTimesTheo(r, 3, [h1,h2,h3]) end, rs)...);
sim2 = vcat(broadcast(function (r) CollisionTimesSim(r, h1, h2, h3) end , rs)...);

nota="Los Resultados Teoricos y Simulados estan en dos tablas.
Primera columna son los radios.
Prmero las bolas entre ellas.
Luego vienen en Teoricas los golpes con las paredes de un bola 
contra una pared asi: 
Columna   Pared
2         x positiva
3         x negativa
4         y positiva
5         y negativa
6         z positiva
7         z negativa
Luego vienen los hops, 
8         hop x
9         hop y
10        hop z
En las simulaciones, entre cada columna, en el mismo orden, hay una columna 
de errores estandar.
"
otranota="La variable h contiene los tamaños de la caja. La variable Nmax el numero de collisiones total."
onde="simulaRosa20201216-1930.jld"

h=(h1,h2,h3)

save(onde, "nota01", nota, "nota02", otranota, 
    "teoricas" , theo, "numericas", sim, "h", h, "Nmax", Nmax)

In [ ]:
using Plots
scatter(rs,[s[1] for s in sim2])
plot!(rs,[t[1] for t in theo2])
title!("Disc collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][1] for s in sim2])
plot!(rs,[t[2][1] for t in theo2])
title!("x wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][3] for s in sim2])
plot!(rs,[t[2][3] for t in theo2])
title!("y wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][5] for s in sim2])
plot!(rs,[t[2][5] for t in theo2])
title!("z wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][1] for s in sim2])
plot!(rs,[t[3][1] for t in theo2])
title!("Hopping x")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][2] for s in sim])
plot!(rs,[t[3][2] for t in theo])
title!("Hopping y")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][3] for s in sim2])
plot!(rs,[t[3][3] for t in theo2])
title!("Hopping z")
xlabel!("r")
ylabel!("mean time")